In [49]:
# !pip install geemap
# !pip install eemont
from google.colab import files
import json
import geemap, ee
ee.Authenticate()
ee.Initialize(project = 'ee-debaratipankaj')


In [50]:
# ------------------------------------------------------
# DEFINE AREA OF INTEREST
# ------------------------------------------------------
#aoi = ee.Geometry.Rectangle([13.2, 52.4, 13.6, 52.6])
print("Upload your GeoJSON file for AOI")
uploaded = files.upload()  # Opens file upload dialog

# Take the first uploaded file
geojson_filename = list(uploaded.keys())[0]
print("Uploaded file:", geojson_filename)

with open(geojson_filename) as f:
    geojson_dict = json.load(f)

# If the GeoJSON is a FeatureCollection, extract the first feature's geometry
if geojson_dict['type'] == 'FeatureCollection':
    geom_dict = geojson_dict['features'][0]['geometry']
else:
    geom_dict = geojson_dict

# Convert GeoJSON geometry to ee.Geometry
roi = ee.Geometry(geom_dict)
print("AOI loaded from GeoJSON.")

Upload your GeoJSON file for AOI


Saving kgp.geojson to kgp.geojson
Uploaded file: kgp.geojson
AOI loaded from GeoJSON.


In [51]:
# roi = ee.Geometry.Polygon(
#     [[[88.30, 22.50],
#       [88.30, 22.65],
#       [88.55, 22.65],
#       [88.55, 22.50]]]
# )


In [52]:
import ee

def mask_s2_clouds(image):
    # Server-side check for QA60 band existence
    qa60_exists = image.bandNames().contains('QA60')

    # Define the QA60 masking logic (Level-1C)
    def qa60_logic(img):
        qa = img.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
               qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        return img.updateMask(mask).divide(10000)

    # Define the MSK_CLASSI masking logic (Level-2A)
    def msk_classi_logic(img):
        mask = img.select('MSK_CLASSI_OPAQUE').eq(0).And(
               img.select('MSK_CLASSI_CIRRUS').eq(0))
        return img.updateMask(mask).divide(10000)

    # Apply server-side conditional logic using ee.Algorithms.If
    return ee.Algorithms.If(
        qa60_exists,
        qa60_logic(image), # If QA60 exists, apply QA60 masking
        msk_classi_logic(image) # Otherwise, apply MSK_CLASSI masking
    )

s2_2018 = ee.ImageCollection("COPERNICUS/S2") \
              .filterBounds(roi) \
              .filterDate('2018-01-01','2018-12-31') \
              .map(mask_s2_clouds) \
              .median()

s2_2025 = ee.ImageCollection("COPERNICUS/S2") \
              .filterBounds(roi) \
              .filterDate('2025-01-01','2025-12-31') \
              .map(mask_s2_clouds) \
              .median()

This modified function now checks for the presence of the `QA60` band. If it exists, it uses the Level-1C cloud masking approach. If `QA60` is not found, it defaults to using `MSK_CLASSI_OPAQUE` and `MSK_CLASSI_CIRRUS` bands, which are common in Level-2A products, to perform cloud masking.

This modified function now checks for the presence of the `QA60` band. If it exists, it uses the Level-1C cloud masking approach. If `QA60` is not found, it defaults to using `MSK_CLASSI_OPAQUE` and `MSK_CLASSI_CIRRUS` bands, which are common in Level-2A products, to perform cloud masking.

In [53]:
def ndbi(img):
    return img.normalizedDifference(['B11', 'B8']).rename('NDBI')

ndbi_2018 = ndbi(s2_2018)
ndbi_2025 = ndbi(s2_2025)


In [54]:
s1_2018 = ee.ImageCollection("COPERNICUS/S1_GRD") \
            .filterBounds(roi) \
            .filterDate('2018-01-01','2018-12-31') \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass','ASCENDING')) \
            .select(['VV','VH']) \
            .median()

s1_2025 = ee.ImageCollection("COPERNICUS/S1_GRD") \
            .filterBounds(roi) \
            .filterDate('2025-01-01','2025-12-31') \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass','ASCENDING')) \
            .select(['VV','VH']) \
            .median()


In [55]:
ratio_2018 = s1_2018.select('VV').divide(s1_2018.select('VH')).rename('RATIO')
ratio_2025 = s1_2025.select('VV').divide(s1_2025.select('VH')).rename('RATIO')


In [56]:
stack_2018 = ndbi_2018.addBands(ratio_2018)
stack_2025 = ndbi_2025.addBands(ratio_2025)


In [57]:
change = stack_2025.subtract(stack_2018).rename(['NDBI_change','SAR_change'])


In [58]:
urban_growth = change.select('NDBI_change').gt(0.1).And(
               change.select('SAR_change').gt(0.5))


In [61]:
Map = geemap.Map()
Map.addLayer(change.select('NDBI_change'),
             {'min': -0.5, 'max': 0.5, 'palette': ['blue','white','red']},
             "Optical Change")

Map.addLayer(change.select('SAR_change'),
             {'min': -3, 'max': 3, 'palette': ['purple','white','yellow']},
             "SAR Change")

Map.addLayer(urban_growth.updateMask(urban_growth),
             {'palette': ['red']}, "Urban Expansion Areas")

Map.centerObject(roi, 11)
Map


Map(center=[22.366979303047774, 87.25713608159592], controls=(WidgetControl(options=['position', 'transparent_…

In [63]:
export = change.toFloat().clip(roi)

task = ee.batch.Export.image.toDrive(
    image=export,
    folder="GEE_Exports",
    description='Urban_Change',
    scale=10,
    region=roi,
    maxPixels=1e13
)
task.start()
